In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras. layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [2]:
TRAIN_DIR='images/train'
TEST_DIR='images/test'

In [3]:
def create_DataFrame(dir):
    img_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            img_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"done")
    return img_paths,labels
        

In [4]:
train= pd.DataFrame()
train['image'],train['label']= create_DataFrame(TRAIN_DIR)

happy done
sad done
fear done
surprise done
neutral done
angry done
disgust done


In [5]:
print(train)

                                image    label
0         images/train/happy/3578.jpg    happy
1        images/train/happy/16988.jpg    happy
2         images/train/happy/2666.jpg    happy
3         images/train/happy/5109.jpg    happy
4        images/train/happy/11981.jpg    happy
...                               ...      ...
28816  images/train/disgust/10112.jpg  disgust
28817  images/train/disgust/21668.jpg  disgust
28818   images/train/disgust/7049.jpg  disgust
28819   images/train/disgust/9716.jpg  disgust
28820   images/train/disgust/3561.jpg  disgust

[28821 rows x 2 columns]


In [6]:
test=pd.DataFrame()
test['image'],test['label']= create_DataFrame(TEST_DIR)

happy done
sad done
fear done
surprise done
neutral done
angry done
disgust done


In [7]:
print(test)

                              image    label
0       images/test/happy/23933.jpg    happy
1       images/test/happy/24906.jpg    happy
2       images/test/happy/18033.jpg    happy
3       images/test/happy/15271.jpg    happy
4       images/test/happy/26888.jpg    happy
...                             ...      ...
7061  images/test/disgust/20761.jpg  disgust
7062  images/test/disgust/28710.jpg  disgust
7063  images/test/disgust/23876.jpg  disgust
7064   images/test/disgust/9460.jpg  disgust
7065  images/test/disgust/35580.jpg  disgust

[7066 rows x 2 columns]


In [8]:
def extract_Features(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image,grayscale=True)
        img= np.array(img)
        features.append(img)
    features= np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features
    
    

In [9]:
train_features= extract_Features(train['image'])


  0%|          | 0/28821 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [10]:
test_features=extract_Features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [11]:
x_train= train_features/255.0

x_test= test_features/255.0



In [12]:
le= LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [13]:
y_train=le.transform(train['label'])
y_test= le.transform(test['label'])
 

In [14]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [15]:
model= Sequential()
#CNN layer
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), metrics=['accuracy'])

model.fit(x= x_train,y = y_train, batch_size = 90, epochs = 50, validation_data = (x_test,y_test)) 



In [ ]:
# model_json= model.to_json()
# with open("Emotion_recog_for_gaming.json",'w') as json_file:
#     json_file.write(model_json)
# model.save("Emotion_recog_for_gaming.h5")

In [23]:
model.fit(x= x_train,y = y_train, batch_size = 64, epochs = 60, validation_data = (x_test,y_test))

Epoch 1/60
451/451 [==============================] - 207s 459ms/step - loss: 1.8219 - accuracy: 0.2454 - val_loss: 1.8096 - val_accuracy: 0.2605
Epoch 2/60
451/451 [==============================] - 247s 548ms/step - loss: 1.7780 - accuracy: 0.2589 - val_loss: 1.6979 - val_accuracy: 0.3125
Epoch 3/60
451/451 [==============================] - 235s 521ms/step - loss: 1.6960 - accuracy: 0.3119 - val_loss: 1.5946 - val_accuracy: 0.3738
Epoch 4/60
451/451 [==============================] - 222s 493ms/step - loss: 1.6073 - accuracy: 0.3675 - val_loss: 1.4969 - val_accuracy: 0.4083
Epoch 5/60
451/451 [==============================] - 220s 487ms/step - loss: 1.5414 - accuracy: 0.3966 - val_loss: 1.4255 - val_accuracy: 0.4367
Epoch 6/60
451/451 [==============================] - 225s 499ms/step - loss: 1.4963 - accuracy: 0.4237 - val_loss: 1.3730 - val_accuracy: 0.4727
Epoch 7/60
451/451 [==============================] - 244s 542ms/step - loss: 1.4563 - accuracy: 0.4314 - val_loss: 1.3199 -

KeyboardInterrupt: 

In [ ]:
model_json= model.to_json()
with open("Emotion_recog_for_gaming.json",'w') as json_file:
    json_file.write(model_json)
model.save("Emotion_recog_for_gaming.h5")